Import needed stuff

In [1]:
import requests
import sys
import pandas as pd
import numpy as np
from scipy import sparse
from io import StringIO
# Stupid funny thing
def progressBar(current, total, barLength = 20):
    percent = float(current) * 100 / total
    arrow   = '-' * int(percent/100 * barLength - 1) + '>'
    spaces  = ' ' * (barLength - len(arrow))
    sys.stdout.write('\rProgress: [%s%s] %d %%' % (arrow, spaces, percent))

Downloading data by http request

In [2]:
reply = requests.get('https://data.europa.eu/euodp/data/storage/f/2020-08-11T140550/Finalised%20mobilities%20started%20in%202018%20-%20KA1.csv')
reply.encoding = 'utf-8'

Transfer in pandas

In [3]:
csv_data = reply.text
df = pd.read_csv(StringIO(csv_data), sep=';',low_memory=False)
for i, name in enumerate(df.columns):
    print(i, name)

0 Project Reference
1 Academic Year
2 Mobility Start Month
3 Mobility End Month
4 Mobility Duration
5 Activity (mob)
6 Field of Education
7 Participant Nationality
8 Education Level
9 Participant Gender
10 Participant Profile
11 Special Needs
12 Fewer Opportunities
13 Participant Age
14 Sending Country Code
15 Sending City
16 Sending Organization
17 Receiving Country Code
18 Receiving City
19 Receiving Organization
20 Participants


## Mobility network
Simply focused on the number of the excange

In [4]:
mobility_df = df[df[df.columns[10]]=='Learner']
mobility_df = mobility_df.loc[:, df.columns[np.array((16, 19, 20))]]
nodes = np.array(list(set.union(set(mobility_df.loc[:, mobility_df.columns[0]]), set(mobility_df.loc[:, mobility_df.columns[1]]))))
# I don't want it to take forever
node_position = {}
for i, name in enumerate(nodes):
    node_position[name] = i
mobility_df['SendId'] = (np.array(list(map(lambda x: node_position[x], mobility_df['Sending Organization']))))
mobility_df['RecId']= (np.array(list(map(lambda x: node_position[x], mobility_df['Receiving Organization']))))
# Adjacency matrix
adj_matrix_crs = sparse.csr_matrix((nodes.size, nodes.size),dtype=int)
adj_matrix_csc = sparse.csc_matrix((nodes.size, nodes.size),dtype=int)
mobility_df

,Sending Organization,Receiving Organization,Participants,SendId,RecId
0,ADAMASTOR - ASSOCIACAO CULTURAL,Stowarzyszenie Jeden Swiat,1,74747,34512
1,ADAMASTOR - ASSOCIACAO CULTURAL,Stowarzyszenie Jeden Swiat,1,74747,34512
53,Landesberufsschule Arnfels,Language Xchange Ireland,1,86878,38862
54,Landesberufsschule Arnfels,Language Xchange Ireland,1,86878,38862
55,Landesberufsschule Arnfels,Language Xchange Ireland,1,86878,38862
...,...,...,...,...,...
688505,Asociación para la integración y Progreso de l...,The National Trust for Places of Historic Inte...,1,48697,90303
688506,COORDINADORA D'ORGANITZADORS DE CAMPS DE TREBA...,VOLUNTEER ACTION FOR PEACE,1,58834,9670
688507,INEX - SDRUZENI DOBROVOLNYCH AKTIVIT Z S,VOLUNTEER ACTION FOR PEACE,1,93748,9670
688508,SOLIDARITES JEUNESSES MCP,VOLUNTEER ACTION FOR PEACE,1,94164,9670


In [5]:
mobility_serie = mobility_df.groupby(['SendId', 'RecId']).sum()
row = np.array(mobility_serie.index.get_level_values(1).tolist())
col = np.array(mobility_serie.index.get_level_values(0).tolist())
val = mobility_serie.values
adj_matrix_crs = sparse.csr_matrix(val, (row, col))

TypeError: only integer scalar arrays can be converted to a scalar index

In [ ]:
l = len(df.index)-1
for i, (index, row) in enumerate(mobility_serie.iterrows()):
    adj_matrix_crs[node_position[index[1]],node_position[index[0]]] = row[0]
    adj_matrix_csc[node_position[index[1]],node_position[index[0]]] = row[0]
    if(i%1000==0):
        progressBar(i, l)
progressBar(l, l)
sparse.save_npz('adj_crs', adj_matrix_crs)
sparse.save_npz('adj_csc', adj_matrix_csc)


Progress: [>                   ] 0 %

C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\sparse\_index.py:82: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])
C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\sparse\_index.py:82: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


Progress: [>                   ] 1 %

KeyboardInterrupt: 

In [ ]:
adj_matrix_crs= sparse.load_npz('adj_crs.npz')
adj_matrix_csc= sparse.load_npz('adj_csc.npz')


In [ ]:
df.groupby('Field of Education').ngroups

168

In [ ]:
print((mobility_serie))

                                                                                                       Participants
Sending Organization                               Receiving Organization                                          
"ADA" University                                   ALMA MATER STUDIORUM - UNIVERSITA DI BOLOGNA                   1
                                                   ANOTATO EKPAIDEYTIKO IDRIMA PEIRAIA TECHNOLOGIK...             2
                                                   Centre d'études Franco Américain de Management                 2
                                                   HUMBOLDT-UNIVERSITAET ZU BERLIN                                2
                                                   ISTANBUL SEHIR UNIVERSITESI                                    2
...                                                                                                             ...
İSTANBUL / BEŞİKTAŞ - Erhan Gedikbaşı Çok Progr... Kindergruppen TEO    